In [1]:
import pandas as pd
import numpy as np

class ETHUpbitStrategy:
    def __init__(self, df):
        self.df = df.copy()
        self.signals = []
        self.position = None
        self.entry_price = 0
        self.entry_atr = 0
        self.bars_in_position = 0
        
    def classify_regime(self, row):
        if row['high_volatility_regime'] == 1 and row['BB_WIDTH'] > self.df['BB_WIDTH'].rolling(20).mean().loc[row.name] * 1.5:
            return "HIGH_VOL"
        elif row['ADX_14'] > 25 and row['TREND_SCORE'] > 0:
            return "TRENDING_UP"
        elif row['ADX_14'] > 25 and row['TREND_SCORE'] < 0:
            return "TRENDING_DOWN"
        elif row['is_squeeze'] == 1:
            return "SQUEEZE"
        else:
            return "RANGING"
    
    def check_long_filters(self, row, regime):
        filters_passed = 0
        score = 0
        
        if regime == "TRENDING_DOWN":
            return False
        
        if row['ADX_14'] < 18:
            return False
        
        if row['Corr_ETH_BTC_24h'] < 0.3:
            return False
        
        if row['TREND_SCORE'] < -0.3:
            return False
        
        if 44 < row['RSI_84'] < 70:
            filters_passed += 1
            score += 1
        
        if row['MACD_12_26_9_Fast_Pct'] > 0 and row['MACD_72_156_54_Slow_Pct'] > 0:
            filters_passed += 1
            score += 2
        
        idx = row.name
        if idx > 0:
            prev_macdh = self.df.loc[idx-1, 'MACDH_12_26_9_Fast_Pct']
            if row['MACDH_12_26_9_Fast_Pct'] > prev_macdh and row['MACDH_12_26_9_Fast_Pct'] > 0:
                filters_passed += 1
                score += 1
        
        if row['CVD_Slope_Norm'] > 0.2:
            filters_passed += 1
            score += 1
        
        if row['MFI_84'] > 52:
            filters_passed += 1
            score += 1
        
        if row['lido_eth_tvl_pct_chg_24h'] > 0 and row['eth_chain_tvl_pct_chg_24h'] > 0:
            filters_passed += 1
            score += 1
        
        if row['RelStr_ETH_BTC'] > 1.02 and row['Corr_ETH_BTC_24h'] > 0.5:
            filters_passed += 1
            score += 2
        
        if row['ETH_KP_Zscore'] > -0.5 and row['ETH_KP_Ratio_slope_180'] > 0:
            filters_passed += 1
            score += 1
        
        if row['VIX_pct_chg_24h'] < 5:
            filters_passed += 1
        
        if row['Div_Bearish'] == 0 and row['Selling_Climax'] == 0:
            filters_passed += 1
        
        if row['REGIME_SMA200'] > 0:
            filters_passed += 1
            score += 1
        
        if row['VOLUME_RATIO'] > 1.0:
            filters_passed += 1
            score += 1
        
        if row['DMP_14'] > row['DMN_14'] * 1.2:
            filters_passed += 1
            score += 1
        
        if regime == "TRENDING_UP":
            required_filters = 8
            required_score = 6
        elif regime == "SQUEEZE":
            required_filters = 8
            required_score = 7
        elif regime == "HIGH_VOL":
            required_filters = 9
            required_score = 8
        elif regime == "RANGING":
            required_filters = 8
            required_score = 6
        else:
            required_filters = 8
            required_score = 6
            
        if filters_passed < required_filters or score < required_score:
            return False
        
        if row['PRICE_VS_LOW_30p'] > 0.3 and row['PRICE_VS_HIGH_30p'] < 0.7 and \
           row['PRICE_VS_LOW_120p'] > 0.25 and \
           abs(row['Dist_from_VWAP']) < 1.2 and \
           row['return_lag_6p'] > -0.005:
            return True
        
        return False
    
    def check_entry_forbidden(self, row):
        if row['fear_greed'] < 15 or row['fear_greed'] > 85:
            return True
        
        if abs(row['Dist_from_VWAP']) > 3:
            return True
        
        if row['VIX_pct_chg_24h'] > 15 and row['VIX_ma180_ratio'] > 1.5:
            return True
        
        if row['RSI_84'] < 25 or row['RSI_84'] > 85:
            return True
        
        return False
    
    def calculate_position_size(self, row, regime):
        base_size = 1.0
        
        if regime == "HIGH_VOL":
            position_size = base_size * 0.5
        elif regime == "SQUEEZE":
            position_size = base_size * 0.7
        elif regime == "TRENDING_UP":
            position_size = base_size * 1.0
        else:
            position_size = base_size * 0.8
        
        if row['NATR_84'] > self.df['NATR_84'].rolling(30).mean().loc[row.name] * 1.5:
            position_size *= 0.7
        
        if row['VOLUME_RATIO'] < 0.8:
            position_size *= 0.8
        
        return position_size
    
    def check_exit_conditions(self, row, regime):
        if row['Selling_Climax'] == 1:
            return True, "SELLING_CLIMAX"
        
        if row['RSI_84'] > 76 or row['MFI_84'] > 80:
            return True, "OVERBOUGHT"
        
        current_return = (row['ETH_Close'] - self.entry_price) / self.entry_price
        
        if current_return < -0.003 and self.bars_in_position > 24:
            return True, "TIME_STOP"
        
        if row['ADX_14'] < 15:
            return True, "ADX_DECLINE"
        
        if row['Corr_ETH_BTC_24h'] < 0.2 and self.bars_in_position > 3:
            return True, "DECORRELATION"
        
        if row['REGIME_SMA200'] < -0.02 and row['TREND_SCORE'] < -0.2:
            return True, "TREND_REVERSAL"
        
        if row['MACD_12_26_9_Fast_Pct'] < -0.3 and row['DMP_14'] < row['DMN_14'] * 1.2:
            return True, "MOMENTUM_REVERSAL"
        
        stop_multiplier = 2.0 if regime == "HIGH_VOL" else 1.6 if row['ADX_14'] > 30 else 1.4
        stop_loss_dist = self.entry_atr * stop_multiplier
        
        if row['ETH_Close'] < self.entry_price - stop_loss_dist:
            return True, "STOP_LOSS"
        
        if row['Vol_Spike'] == 1 and current_return < -0.005 and self.bars_in_position > 1:
            return True, "VOL_SPIKE_STOP"
        
        if current_return > 0.012:
            highest = self.df.loc[self.df.index[self.df.index.get_loc(row.name) - self.bars_in_position]:row.name, 'ETH_High'].max()
            trailing_stop = max(self.entry_price + self.entry_atr * 0.6, highest - self.entry_atr * 1.6)
            if row['ETH_Close'] < trailing_stop:
                return True, "TRAILING_STOP"
        elif current_return > 0.006 and self.bars_in_position > 6:
            highest = self.df.loc[self.df.index[self.df.index.get_loc(row.name) - self.bars_in_position]:row.name, 'ETH_High'].max()
            trailing_stop = self.entry_price + (highest - self.entry_price) * 0.4
            if row['ETH_Close'] < trailing_stop:
                return True, "TRAILING_STOP"
        
        tp1 = self.entry_atr * 1.5
        tp2 = self.entry_atr * 3.0
        tp3 = self.entry_atr * 5.0
        
        if current_return > tp3 / self.entry_price:
            return True, "TP3"
        
        return False, None
    
    def run_backtest(self):
        for idx in range(30, len(self.df)):
            row = self.df.iloc[idx]
            regime = self.classify_regime(row)
            
            if self.position is not None:
                self.bars_in_position += 1
                should_exit, exit_reason = self.check_exit_conditions(row, regime)
                
                if should_exit:
                    exit_price = row['ETH_Close']
                    pnl = (exit_price - self.entry_price) / self.entry_price * 100
                    
                    self.signals.append({
                        'date': row['date'],
                        'action': 'EXIT',
                        'price': exit_price,
                        'pnl': pnl,
                        'reason': exit_reason,
                        'bars_held': self.bars_in_position,
                        'regime': regime
                    })
                    
                    self.position = None
                    self.entry_price = 0
                    self.entry_atr = 0
                    self.bars_in_position = 0
            
            else:
                if self.check_entry_forbidden(row):
                    continue
                
                if self.check_long_filters(row, regime):
                    position_size = self.calculate_position_size(row, regime)
                    self.position = "LONG"
                    self.entry_price = row['ETH_Close']
                    self.entry_atr = row['ATR_84']
                    self.bars_in_position = 0
                    
                    self.signals.append({
                        'date': row['date'],
                        'action': 'ENTRY',
                        'price': self.entry_price,
                        'regime': regime,
                        'position_size': position_size,
                        'atr': self.entry_atr,
                        'rsi': row['RSI_84'],
                        'adx': row['ADX_14']
                    })
        
        return pd.DataFrame(self.signals)


def analyze_results(signals_df):
    if len(signals_df) == 0:
        return "신호 없음"
    
    exits = signals_df[signals_df['action'] == 'EXIT']
    entries = signals_df[signals_df['action'] == 'ENTRY']
    
    if len(exits) == 0:
        return f"진입 신호: {len(entries)}개, 아직 청산 없음"
    
    total_trades = len(exits)
    winning_trades = len(exits[exits['pnl'] > 0])
    losing_trades = len(exits[exits['pnl'] <= 0])
    
    win_rate = winning_trades / total_trades * 100 if total_trades > 0 else 0
    avg_pnl = exits['pnl'].mean()
    total_pnl = exits['pnl'].sum()
    max_win = exits['pnl'].max()
    max_loss = exits['pnl'].min()
    avg_bars_held = exits['bars_held'].mean()
    
    avg_win = exits[exits['pnl'] > 0]['pnl'].mean() if winning_trades > 0 else 0
    avg_loss = exits[exits['pnl'] <= 0]['pnl'].mean() if losing_trades > 0 else 0
    profit_factor = abs(avg_win * winning_trades / (avg_loss * losing_trades)) if losing_trades > 0 and avg_loss != 0 else 0
    
    exit_reasons = exits['reason'].value_counts()
    
    result = f"""
백테스트 결과:
================
총 거래: {total_trades}
승리: {winning_trades} ({win_rate:.2f}%)
패배: {losing_trades}

수익 지표:
평균 수익률: {avg_pnl:.2f}%
누적 수익률: {total_pnl:.2f}%
최대 수익: {max_win:.2f}%
최대 손실: {max_loss:.2f}%
평균 승리: {avg_win:.2f}%
평균 패배: {avg_loss:.2f}%
Profit Factor: {profit_factor:.2f}

보유 시간:
평균: {avg_bars_held:.1f}봉 ({avg_bars_held*4:.1f}시간)

청산 사유:
"""
    for reason, count in exit_reasons.items():
        result += f"{reason}: {count}회\n"
    
    return result

In [2]:
import pandas as pd
df=pd.read_csv("eth_4hour_all.csv")
df['date']=pd.to_datetime(df['date'])
df=df[df['date']>="2018-01-01"]
strategy = ETHUpbitStrategy(df)
signals = strategy.run_backtest()
print(analyze_results(signals))

신호 없음
